In [9]:
import pandas as pd
from dotenv import load_dotenv
import os
from tasc_core.utils.util_nebuladb import NebulaConnector
from tasc_core.utils.util_shopify_product_parser import ShopifyProductParser
from requests.exceptions import HTTPError  # Add this import
import requests


# Load environment variables from .env file
load_dotenv()

username = os.getenv('NEBULA_DB_USERNAME')
password = os.getenv('NEBULA_DB_PASSWORD')

# Get the current working directory
project_path = os.getcwd()

print("Project Path:", project_path)


Project Path: /Users/oladiranadanijo/github_repos/tasc_core/test


In [24]:
nebula = NebulaConnector()
shopify_products_df = nebula.select_df("SELECT * FROM tasc_prod.tasc_xref_partners LIMIT 10")

In [13]:
# Specify the encoding to handle non-UTF-8 characters
# test_df = pd.read_csv('/Users/oladiranadanijo/github_repos/tasc_core/resources/tasc_xref_partners_seed.csv', encoding='ISO-8859-1')

In [14]:
# nebula.insert_df(table_name='tasc_xref_partners', table_schema='tasc_prod', df=test_df)

In [15]:
import requests
import json

def read_json_from_url(url):
    response = requests.get(url)
    response.raise_for_status()  # Raise an exception for HTTP errors
    return response.json()

# Example usage
url = 'https://www.houseoferrors.org/products.json'
json_data = read_json_from_url(url)

# Pretty-print JSON data with default indent value of 4
# print(json.dumps(json_data, indent=4))

# Print JSON data without specifying indent (compact format)
# print(json.dumps(json_data))

In [21]:
# Assuming partners_df is already defined and contains a column named 'partner_url'
for index, row in partners_df.iterrows():
    print(row['partner_primary_url'])
    # Create an instance of the parser
    parser = ShopifyProductParser(row['partner_primary_url'] + 'products.json')

    try:
        # Load and parse the JSON data
        parser.load_json()
        parsed_df = parser.to_dataframe()

        # Assuming parsed_df is your DataFrame
        for column in parsed_df.columns:
            parsed_df[column] = parsed_df[column].astype(str).str[:255]

        # Convert all columns to object data type
        # parsed_df = parsed_df.astype('object')

        # Display the DataFrame with every field captured
        display(parsed_df)
        print(parsed_df.columns)
        print(parsed_df.dtypes)
        print(len(parsed_df))

        # Print the distinct count of product_title
        distinct_product_titles = parsed_df['handle'].nunique()
        print(f"Distinct count of product_title: {distinct_product_titles}")

        # Insert the DataFrame into the database table
        parsed_df['asof_dt'] = pd.to_datetime('today').date()  # Add asof_dt column
        parsed_df['created_at_tms'] = pd.to_datetime('now')  # Add created_at_tms column
        parsed_df['created_by'] = 'oadanijo'  # Add created_by column
        parsed_df['last_modified_tms'] = pd.to_datetime('now')  # Add last_modified_tms column
        parsed_df['last_modified_by'] = 'oadanijo'  # Add last_modified_by column
        parsed_df['source_date'] = pd.to_datetime('today').date()  # Add source_date column
        parsed_df['source_datetime'] = pd.to_datetime('now')  # Add source_datetime column

        nebula.insert_df(table_name='tasc_products_shopify', table_schema='tasc_prod', df=parsed_df)

    except ValueError as e:
        print(f"Error processing {row['partner_primary_url']}: {e}")
    except HTTPError as e:
        print(f"HTTP error processing {row['partner_primary_url']}: {e}")
    except Exception as e:
        print(f"Unexpected error processing {row['partner_primary_url']}: {e}")
    # break

In [20]:
partners_df

,id,asof_dt,partner_uid,partner_name,partner_full_name,partner_desc,partner_type,partner_primary_url,created_at_tms,created_by,last_modified_tms,last_modified_by,source_date,source_datetime
0,1,2024-11-09,1,Houser of Errors,None,None,VENDOR,https://www.houseoferrors.org/,None,oadanijo,2024-11-09,oadanijo,2024-11-09,2024-11-09
1,2,2024-11-09,2,Sage NationÊ,None,None,VENDOR,https://sagenation.uk/,None,oadanijo,2024-11-09,oadanijo,2024-11-09,2024-11-09
2,3,2024-11-09,3,Abaga,Abaga Velli,None,VENDOR,https://abagavelli.com/,None,oadanijo,2024-11-09,oadanijo,2024-11-09,2024-11-09
3,4,2024-11-09,4,A day without angstÊ,A day without angstÊ,None,VENDOR,https://adwoa.info/,None,oadanijo,2024-11-09,oadanijo,2024-11-09,2024-11-09
4,5,2024-11-09,5,Two JeysÊ,None,None,VENDOR,https://twojeys.com/,None,oadanijo,2024-11-09,oadanijo,2024-11-09,2024-11-09
5,6,2024-11-09,6,Serge DeNimes,None,None,VENDOR,https://www.sergedenimes.com/,None,oadanijo,2024-11-09,oadanijo,2024-11-09,2024-11-09
6,7,2024-11-09,7,CLINTS Inc,None,None,VENDOR,https://clints.co/,None,oadanijo,2024-11-09,oadanijo,2024-11-09,2024-11-09
7,8,2024-11-09,8,Corteiz,None,None,VENDOR,https://www.crtz.xyz/,None,oadanijo,2024-11-09,oadanijo,2024-11-09,2024-11-09
8,9,2024-11-09,9,Carsico,None,None,VENDOR,https://www.carsick-o.com/,None,oadanijo,2024-11-09,oadanijo,2024-11-09,2024-11-09
9,10,2024-11-09,10,kortsiide,None,None,VENDOR,https://www.kortsiide.com/,None,oadanijo,2024-11-09,oadanijo,2024-11-09,2024-11-09


In [27]:
nebula = NebulaConnector()
shopify_products_df = nebula.select_df("SELECT * FROM tasc_prod.tasc_products_shopify LIMIT 10")

In [28]:
shopify_products_df

,id,parent_product_id,child_product_id,product_title,product_desc,handle,vendor,product_type,tags,published_at,...,product_image_3_url,product_image_4_url,product_image_5_url,asof_dt,created_at_tms,created_by,last_modified_tms,last_modified_by,source_date,source_datetime
0,1,8581629051043,45156926685347,KNITTED MOTO TOP,Knitted Polo Top from soft Acrylic/Nylon Yarn ...,knitted-moto-top,HOUSE OF ERRORS,None,None,2024-09-01 16:20:43,...,https://cdn.shopify.com/s/files/1/0327/7452/09...,https://cdn.shopify.com/s/files/1/0327/7452/09...,None,2024-09-12,2024-09-12 06:53:12.050412,oadanijo,2024-09-12 06:53:12.050632,oadanijo,2024-09-12,2024-09-12 06:53:12.050894
1,2,8581629051043,45156926718115,KNITTED MOTO TOP,Knitted Polo Top from soft Acrylic/Nylon Yarn ...,knitted-moto-top,HOUSE OF ERRORS,None,None,2024-09-01 16:20:43,...,https://cdn.shopify.com/s/files/1/0327/7452/09...,https://cdn.shopify.com/s/files/1/0327/7452/09...,None,2024-09-12,2024-09-12 06:53:12.050412,oadanijo,2024-09-12 06:53:12.050632,oadanijo,2024-09-12,2024-09-12 06:53:12.050894
2,3,8581629051043,45156926750883,KNITTED MOTO TOP,Knitted Polo Top from soft Acrylic/Nylon Yarn ...,knitted-moto-top,HOUSE OF ERRORS,None,None,2024-09-01 16:20:43,...,https://cdn.shopify.com/s/files/1/0327/7452/09...,https://cdn.shopify.com/s/files/1/0327/7452/09...,None,2024-09-12,2024-09-12 06:53:12.050412,oadanijo,2024-09-12 06:53:12.050632,oadanijo,2024-09-12,2024-09-12 06:53:12.050894
3,4,8581629051043,45156926783651,KNITTED MOTO TOP,Knitted Polo Top from soft Acrylic/Nylon Yarn ...,knitted-moto-top,HOUSE OF ERRORS,None,None,2024-09-01 16:20:43,...,https://cdn.shopify.com/s/files/1/0327/7452/09...,https://cdn.shopify.com/s/files/1/0327/7452/09...,None,2024-09-12,2024-09-12 06:53:12.050412,oadanijo,2024-09-12 06:53:12.050632,oadanijo,2024-09-12,2024-09-12 06:53:12.050894
4,5,8581038997667,45155570188451,KNITTED MOTO JACKET,"<p>Knitted Jacket from soft <meta charset=""utf...",knitted-moto-jacket,HOUSE OF ERRORS,None,None,2024-09-01 16:19:30,...,https://cdn.shopify.com/s/files/1/0327/7452/09...,https://cdn.shopify.com/s/files/1/0327/7452/09...,https://cdn.shopify.com/s/files/1/0327/7452/09...,2024-09-12,2024-09-12 06:53:12.050412,oadanijo,2024-09-12 06:53:12.050632,oadanijo,2024-09-12,2024-09-12 06:53:12.050894
5,6,8581038997667,45155570221219,KNITTED MOTO JACKET,"<p>Knitted Jacket from soft <meta charset=""utf...",knitted-moto-jacket,HOUSE OF ERRORS,None,None,2024-09-01 16:19:30,...,https://cdn.shopify.com/s/files/1/0327/7452/09...,https://cdn.shopify.com/s/files/1/0327/7452/09...,https://cdn.shopify.com/s/files/1/0327/7452/09...,2024-09-12,2024-09-12 06:53:12.050412,oadanijo,2024-09-12 06:53:12.050632,oadanijo,2024-09-12,2024-09-12 06:53:12.050894
6,7,8581038997667,45155570253987,KNITTED MOTO JACKET,"<p>Knitted Jacket from soft <meta charset=""utf...",knitted-moto-jacket,HOUSE OF ERRORS,None,None,2024-09-01 16:19:30,...,https://cdn.shopify.com/s/files/1/0327/7452/09...,https://cdn.shopify.com/s/files/1/0327/7452/09...,https://cdn.shopify.com/s/files/1/0327/7452/09...,2024-09-12,2024-09-12 06:53:12.050412,oadanijo,2024-09-12 06:53:12.050632,oadanijo,2024-09-12,2024-09-12 06:53:12.050894
7,8,8581038997667,45155570286755,KNITTED MOTO JACKET,"<p>Knitted Jacket from soft <meta charset=""utf...",knitted-moto-jacket,HOUSE OF ERRORS,None,None,2024-09-01 16:19:30,...,https://cdn.shopify.com/s/files/1/0327/7452/09...,https://cdn.shopify.com/s/files/1/0327/7452/09...,https://cdn.shopify.com/s/files/1/0327/7452/09...,2024-09-12,2024-09-12 06:53:12.050412,oadanijo,2024-09-12 06:53:12.050632,oadanijo,2024-09-12,2024-09-12 06:53:12.050894
8,9,8581038997667,45155570319523,KNITTED MOTO JACKET,"<p>Knitted Jacket from soft <meta charset=""utf...",knitted-moto-jacket,HOUSE OF ERRORS,None,None,2024-09-01 16:19:30,...,https://cdn.shopify.com/s/files/1/0327/7452/09...,https://cdn.shopify.com/s/files/1/0327/7452/09...,https://cdn.shopify.com/s/files/1/0327/7452/09...,2024-09-12,2024-09-12 06:53:12.050412,oada

In [29]:
# Function to download an image from a URL and save it to a file
def download_image(url, file_path):
    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise an exception for HTTP errors
        with open(file_path, 'wb') as f:
            f.write(response.content)
        print(f"Downloaded {file_path}")
    except Exception as e:
        print(f"Failed to download {url}: {e}")

# Loop over partners_df and download images
for index, row in shopify_products_df.iterrows():
    parent_product_id = row['parent_product_id']
    vendor = row['vendor']
    image_urls = [
        row['product_image_1_url'],
        row['product_image_2_url'],
        row['product_image_3_url'],
        row['product_image_4_url'],
        row['product_image_5_url']
    ]

    for i, url in enumerate(image_urls, start=1):
        if pd.notna(url):  # Check if the URL is not NaN
            file_name = f"/output_images_test/{parent_product_id}_{vendor}_image_{i}.jpg"
            download_image(url, file_name)

Failed to download https://cdn.shopify.com/s/files/1/0327/7452/0971/files/IMG_1949.jpg?v=1725126033: [Errno 2] No such file or directory: '/output_images_test/8581629051043_HOUSE OF ERRORS_image_1.jpg'
Failed to download https://cdn.shopify.com/s/files/1/0327/7452/0971/files/IMG_1946.jpg?v=1725126031: [Errno 2] No such file or directory: '/output_images_test/8581629051043_HOUSE OF ERRORS_image_2.jpg'
Failed to download https://cdn.shopify.com/s/files/1/0327/7452/0971/files/IMG_1948.jpg?v=1725126039: [Errno 2] No such file or directory: '/output_images_test/8581629051043_HOUSE OF ERRORS_image_3.jpg'


KeyboardInterrupt: 